In [1]:
import os, sys, gzip
import numpy as np
from six.moves import cPickle
sys.setrecursionlimit(10000)

from lasagne import layers, nonlinearities, updates, objectives, init 
from lasagne.layers import get_output, get_output_shape, get_all_params
import theano.tensor as T
import theano
from theano.tensor.shared_randomstreams import RandomStreams
from lasagne.layers import Conv2DLayer, DenseLayer, InputLayer, ExpressionLayer, BiasLayer
from lasagne.layers import NonlinearityLayer, MaxPool2DLayer, DropoutLayer, BatchNormLayer


sys.path.append('..')
from src import NeuralNet
from src import train as fit
from src import make_directory 
from models import load_model
from data import load_data

from subprocess import call

np.random.seed(247) # for reproducibility

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
%matplotlib inline
from scipy.misc import imresize

datadir = '/home/peter/Data/SequenceMotif/networkcomparison'
savedir = make_directory(datadir, 'deep')

from tsne import bh_sne
from subprocess import call

Using gpu device 0: GeForce GTX 980 (CNMeM is disabled, CuDNN 4007)


In [2]:
filename = 'Localized_N=100000_S=200_M=50_G=20_data.pickle'
datapath = '/home/peter/Data/SequenceMotif'
filepath = os.path.join(datapath, filename)

# load training set
print "loading data from: " + filepath
f = open(filepath, 'rb')
print "loading train data"
train = cPickle.load(f)
print "loading cross-validation data"
cross_validation = cPickle.load(f)
print "loading test data"
test = cPickle.load(f)
f.close()

X_train = train[0].transpose((0,1,2)).astype(np.float32)
y_train = train[1].astype(np.int32)
X_val = cross_validation[0].transpose((0,1,2)).astype(np.float32)
y_val = cross_validation[1].astype(np.int32)
X_test = test[0].transpose((0,1,2)).astype(np.float32)
y_test = test[1].astype(np.int32)

X_train = np.expand_dims(X_train, axis=3)
X_val = np.expand_dims(X_val, axis=3)
X_test = np.expand_dims(X_test, axis=3)

train = (X_train, y_train, train[2])
valid = (X_val, y_val, cross_validation[2])
test = (X_test, y_test, test[2])

shape = (None, train[0].shape[1], train[0].shape[2], train[0].shape[3])
num_labels = train[1].shape[1]

loading data from: /home/peter/Data/SequenceMotif/Unlocalized_N=100000_S=200_M=50_G=20_data.pickle
loading train data
loading cross-validation data
loading test data


In [ ]:
"""model_name = "deepsea_model"
nnmodel = NeuralNet(model_name, shape, num_labels)

nnmodel = fit.train_minibatch(nnmodel, train, valid, batch_size=128, num_epochs=500, 
                        patience=3, verbose=1, filepath=filepath)

# save best model --> lowest cross-validation error
min_loss, min_index = nnmodel.get_min_loss()
savepath = filepath + "_epoch_" + str(min_index) + ".pickle"
nnmodel.set_parameters_from_file(savepath)

# test model
nnmodel.test_model(test, 512, "test")

In [ ]:
model_name = "autoencode_motif_model"
nnmodel = NeuralNet(model_name, shape, num_labels)

nnmodel = fit.train_minibatch(nnmodel, train, valid, batch_size=128, num_epochs=500, 
                        patience=3, verbose=1, filepath=filepath)

# save best model --> lowest cross-validation error
min_loss, min_index = nnmodel.get_min_loss()
savepath = filepath + "_epoch_" + str(min_index) + ".pickle"
nnmodel.set_parameters_from_file(savepath)

# test model
nnmodel.test_model(test, 512, "test")

In [3]:
model_name = "test_motif_model"
nnmodel = NeuralNet(model_name, shape, num_labels)

nnmodel = fit.train_minibatch(nnmodel, train, valid, batch_size=128, num_epochs=500, 
                        patience=3, verbose=1, filepath=filepath)

# save best model --> lowest cross-validation error
min_loss, min_index = nnmodel.get_min_loss()
savepath = filepath + "_epoch_" + str(min_index) + ".pickle"
nnmodel.set_parameters_from_file(savepath)

# test model
nnmodel.test_model(test, 512, "test")


Epoch 1 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.41746 -- accuracy=96.34%  
  valid loss:		0.28745
  valid accuracy:	0.96704+/-0.02111
  valid auc-roc:	0.92560+/-0.06341
  valid auc-pr:		0.58827+/-0.29510
saving model parameters to: /home/peter/Data/SequenceMotif/Unlocalized_N=100000_S=200_M=50_G=20_data.pickle_epoch_0.pickle
Epoch 2 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.23668 -- accuracy=97.48%  
  valid loss:		0.22797
  valid accuracy:	0.97320+/-0.01963
  valid auc-roc:	0.95153+/-0.04403
  valid auc-pr:		0.68397+/-0.25874
saving model parameters to: /home/peter/Data/SequenceMotif/Unlocalized_N=100000_S=200_M=50_G=20_data.pickle_epoch_1.pickle
Epoch 3 out of 500 
[==============================] 100.0% -- time=0s -- loss=0.20240 -- accuracy=97.77%  
  valid loss:		0.19460
  valid accuracy:	0.97595+/-0.01592
  valid auc-roc:	0.96237+/-0.03649
  valid auc-pr:		0.72789+/-0.24032
saving model parameters to: /home/peter/Dat

0.082959803369206456

In [ ]:
58/2
